In [1]:
import requests
import pandas as pd
import sys
import json

In [7]:
allTickers = json.loads(open("tickers.json", 'r').read())
ticker = "IBM"
cik = allTickers[ticker]['cik']

In [8]:
user_agent = {"user-agent": "www.jonsteeves.dev jonathonsteeves@cmail.carleton.ca"}
url = "https://www.sec.gov/edgar/browse/?CIK=51143"
companyFacts = "https://data.sec.gov/api/xbrl/companyfacts/CIK{:010d}.json".format(cik)
api = "https://data.sec.gov/submissions/CIK0000051143.json"

In [9]:
companyFacts

'https://data.sec.gov/api/xbrl/companyfacts/CIK0000051143.json'

In [6]:
res = requests.get(companyFacts, headers=user_agent)
res.status_code

200

In [23]:
dc = dict(res.json())

In [24]:
dc['facts'].keys()

dict_keys(['dei', 'us-gaap'])

In [26]:
len(dc['facts']['us-gaap'])

713

In [27]:
sys.getsizeof(dc["facts"]['us-gaap'])

20536

In [86]:
dc['facts']['us-gaap']['Revenues']

{'label': 'Revenues',
 'description': 'Amount of revenue recognized from goods sold, services rendered, insurance premiums, or other activities that constitute an earning process. Includes, but is not limited to, investment and interest income before deduction of interest expense when recognized as a component of revenue, and sales and trading gain (loss).',
 'units': {'USD': [{'start': '2007-01-01',
    'end': '2007-12-31',
    'val': 98786000000,
    'accn': '0001047469-10-001151',
    'fy': 2009,
    'fp': 'FY',
    'form': '10-K',
    'filed': '2010-02-23',
    'frame': 'CY2007'},
   {'start': '2008-01-01',
    'end': '2008-06-30',
    'val': 51322000000,
    'accn': '0001104659-09-045198',
    'fy': 2009,
    'fp': 'Q2',
    'form': '10-Q',
    'filed': '2009-07-28'},
   {'start': '2008-04-01',
    'end': '2008-06-30',
    'val': 26820000000,
    'accn': '0001104659-09-045198',
    'fy': 2009,
    'fp': 'Q2',
    'form': '10-Q',
    'filed': '2009-07-28',
    'frame': 'CY2008Q2'},

In [74]:
dc['facts']['dei']['EntityPublicFloat']

{'label': 'Entity Public Float',
 'description': "The aggregate market value of the voting and non-voting common equity held by non-affiliates computed by reference to the price at which the common equity was last sold, or the average bid and asked price of such common equity, as of the last business day of the registrant's most recently completed second fiscal quarter.",
 'units': {'USD': [{'end': '2008-06-30',
    'val': 160600000000,
    'accn': '0001104659-09-045198',
    'fy': 2009,
    'fp': 'Q2',
    'form': '10-Q',
    'filed': '2009-07-28'},
   {'end': '2008-06-30',
    'val': 160600000000,
    'accn': '0001104659-09-060554',
    'fy': 2009,
    'fp': 'Q3',
    'form': '10-Q',
    'filed': '2009-10-27',
    'frame': 'CY2008Q2I'},
   {'end': '2009-06-30',
    'val': 136800000000,
    'accn': '0001047469-10-001151',
    'fy': 2009,
    'fp': 'FY',
    'form': '10-K',
    'filed': '2010-02-23',
    'frame': 'CY2009Q2I'},
   {'end': '2010-06-30',
    'val': 155700000000,
    'accn

In [81]:
dc['facts']['dei'].keys()

dict_keys(['EntityCommonStockSharesOutstanding', 'EntityPublicFloat'])

In [28]:
gaap = dc['facts']['us-gaap']

In [69]:
gaap["AccruedLiabilitiesAndOtherLiabilitiesCurrent"]['units']['USD'][-3:]

KeyError: 'AccruedLiabilitiesAndOtherLiabilitiesCurrent'

In [12]:
equityFinancials = {}

In [61]:
gaap["IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest"]['units']['USD'][-2]['val'] + gaap["InterestExpense"]['units']['USD'][-2]['val']

1610000000

In [106]:
description = {}
for i in gaap.keys():
    description[i] = {"description": gaap[i]["description"]}

In [107]:
with open("us-gaap-descriptions.json", 'w') as file:
    file.write(json.dumps(description))

In [25]:
keyAttributes = {}
with open("keyAttributes.json", 'r') as file:
    lines = file.read()
    keyAttributes = json.loads(lines)
    

In [26]:
keyAttributes

{'Services': 'CostOfRevenue',
 'Sales': 'CostOfRevenue',
 'Financing': 'CostOfRevenue',
 'Total revenue': 'Revenues',
 'Total cost': 'CostOfRevenue',
 'Gross profit': 'GrossProfit',
 'Selling, general and administrative': 'SellingGeneralAndAdministrativeExpense',
 'Research, development and engineering': 'ResearchAndDevelopmentExpense',
 'Intellectual property and custom development income': 'IntellectualPropertyAndCustomDevelopmentIncome',
 'Other (income) and expense': 'OtherIncomeAndExpense',
 'Interest expense': 'InterestExpense',
 'Total expense and other (income)': 'ExpenseAndOtherIncome',
 'Income from continuing operations before income taxes': 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest',
 'Provision for/(benefit from) income taxes': 'IncomeTaxExpenseBenefit',
 'Income from continuing operations': 'IncomeLossFromContinuingOperations',
 'Net income': 'NetIncomeLoss',
 'EPS Diluted Total': 'EarningsPerShareDiluted',
 'EPS Basic To

In [58]:
gaap2 = gaap

In [153]:
gaap["AccountsNotesAndLoansReceivableNetCurrent"]['label']

'Accounts and Financing Receivable, after Allowance for Credit Loss, Current'

In [65]:
keys = list(gaap.keys())
for g in keys:
    if(gaap2[g]['label'] is None):
        continue
    if('Deprecated' in gaap2[g]['label']):
        gaap2.pop(g);

In [170]:
description.keys()

NameError: name 'description' is not defined

In [138]:
things = set(gaap2.keys())
things

{'AccountsNotesAndLoansReceivableNetCurrent',
 'AccountsPayableCurrent',
 'AccountsReceivableNetCurrent',
 'AccrualForEnvironmentalLossContingencies',
 'AccruedEnvironmentalLossContingenciesNoncurrent',
 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment',
 'AccumulatedOtherComprehensiveIncomeLossNetOfTax',
 'AdjustmentsRelatedToTaxWithholdingForShareBasedCompensation',
 'AllocatedShareBasedCompensationExpense',
 'AllocatedShareBasedCompensationExpenseNetOfTax',
 'AllowanceForDoubtfulAccountsReceivable',
 'AllowanceForDoubtfulAccountsReceivableCurrent',
 'AllowanceForDoubtfulAccountsReceivablePeriodIncreaseDecrease',
 'AllowanceForDoubtfulAccountsReceivableWriteOffs',
 'AllowanceForDoubtfulOtherReceivablesCurrent',
 'AllowanceForNotesAndLoansReceivableCurrent',
 'AllowanceForNotesAndLoansReceivableNoncurrent',
 'AmortizationOfFinancingCosts',
 'AmortizationOfIntangibleAssets',
 'AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount',
 'AssetRetire

In [ ]:
'CostMethodInvestments',
'CostOfGoodsSold',
'CostOfGoodsSoldSalesTypeLease',
'CostOfRevenue',
'CostOfServices',

In [27]:
for val in keyAttributes.values():
    if(val not in things):
        print(val)

IntellectualPropertyAndCustomDevelopmentIncome
OtherIncomeAndExpense
ExpenseAndOtherIncome
None
AccruedLiabilitiesAndOtherLiabilitiesCurrent
GainLossOnDispositionOfAssetsAndOther
PaymentsToAcquireAndDevelopSoftware
NonOperatingFinanceReceivablesNet


In [157]:
gaap["GrossProfit"]["units"]["USD"][-2:-1]

[{'start': '2021-07-01',
  'end': '2021-09-30',
  'val': 8171000000,
  'accn': '0001558370-21-014734',
  'fy': 2021,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2021-11-05',
  'frame': 'CY2021Q3'}]

In [163]:
gaap["GrossProfit"]["units"]["USD"][-2:-1][0]['val'] - gaap["NetIncomeLoss"]["units"]["USD"][-2:-1][0]['val']

7041000000

In [80]:
dataModel = json.loads(open('dataModelV1.json', 'r').read())
gaapKeys = set(gaap.keys())

In [81]:
for d in dataModel.values():
    if(d in gaapKeys):
        print(gaap[d]['label'])
#     units = gaap[g]["units"]
#     if("USD" in units.keys()):

Revenues
Cost of Revenue
Gross Profit
Selling, General and Administrative Expense
Research and Development Expense
Interest Expense
Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest
Income Tax Expense (Benefit)
Net Income (Loss) Attributable to Parent
Earnings Per Share, Diluted
Earnings Per Share, Basic
Weighted Average Number of Shares Outstanding, Diluted
Weighted Average Number of Shares Outstanding, Basic
Other Comprehensive Income (Loss), Foreign Currency Transaction and Translation Adjustment, before Tax, Portion Attributable to Parent
Other Comprehensive Income (Loss), Available-for-sale Securities Adjustment, before Tax, Portion Attributable to Parent
Other Comprehensive Income (Loss), Cash Flow Hedge, Gain (Loss), after Reclassification, before Tax, Parent
Other Comprehensive (Income) Loss, Defined Benefit Plan, before Tax, after Reclassification Adjustment, Attributable to Parent
Other Comprehensive Income (Loss), before Tax, Portion Attri

In [119]:
def keyQuarters(ls):
    newls = []
#     ytd = 0
    for frame in ls:
        if('frame' in frame.keys()):
#             print('Q' in frame['frame'])
#             print(frame)
            newls.append(frame)
#             if('Q' in frame['frame']):
# #                 print(frame)
#                 ytd += int(frame['val'])
#             else:
#                 print(ytd, "-", frame['val'], ytd-int(frame['val']))
#                 ytd = 0
    return newls
        

In [121]:
qs = keyQuarters(gaap["Revenues"]['units']['USD'])

In [133]:
qs[0]

{'start': '2007-01-01',
 'end': '2007-12-31',
 'val': 98786000000,
 'accn': '0001047469-10-001151',
 'fy': 2009,
 'fp': 'FY',
 'form': '10-K',
 'filed': '2010-02-23',
 'frame': 'CY2007'}

In [134]:
df = pd.DataFrame(qs).set_index('end')

In [135]:
df[['fp', 'val']].transpose()

end,2007-12-31,2008-06-30,2008-09-30,2008-12-31,2009-03-31,2009-06-30,2009-09-30,2009-12-31,2010-03-31,2010-06-30,...,2019-09-30,2019-12-31,2020-03-31,2020-06-30,2020-09-30,2020-12-31,2021-03-31,2021-06-30,2021-09-30,2021-12-31
fp,FY,Q2,Q3,FY,Q1,Q2,Q3,FY,Q1,Q2,...,Q3,FY,Q1,Q2,Q3,FY,Q1,Q2,Q3,FY
val,98786000000,26820000000,25302000000,103630000000,21711000000,23250000000,23566000000,95758000000,22857000000,23724000000,...,18028000000,57714000000,17571000000,18123000000,17560000000,55179000000,17730000000,18745000000,17618000000,57350000000


In [88]:
gaap["Revenues"]['units']['USD']

[{'start': '2007-01-01',
  'end': '2007-12-31',
  'val': 98786000000,
  'accn': '0001047469-10-001151',
  'fy': 2009,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2010-02-23',
  'frame': 'CY2007'},
 {'start': '2008-01-01',
  'end': '2008-06-30',
  'val': 51322000000,
  'accn': '0001104659-09-045198',
  'fy': 2009,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2009-07-28'},
 {'start': '2008-04-01',
  'end': '2008-06-30',
  'val': 26820000000,
  'accn': '0001104659-09-045198',
  'fy': 2009,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2009-07-28',
  'frame': 'CY2008Q2'},
 {'start': '2008-01-01',
  'end': '2008-09-30',
  'val': 76623000000,
  'accn': '0001104659-09-060554',
  'fy': 2009,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2009-10-27'},
 {'start': '2008-07-01',
  'end': '2008-09-30',
  'val': 25302000000,
  'accn': '0001104659-09-060554',
  'fy': 2009,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2009-10-27',
  'frame': 'CY2008Q3'},
 {'start': '2008-01-01',
  'end': '2008-12-31',
  'va